# Code to inference model to recoginize different hand signs as letters.

In [1]:
import cv2
import os
import pickle
import mediapipe as mp
import numpy as np

In [2]:
# Load model for inferencing
model_data = pickle.load(open('model.p', 'rb'))
model = model_data[0]

In [3]:
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [ ]:
while True:
    data_aux, x_, y_ = [], [], []

    ret, frame = cap.read()
    H, W, channel = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Drawing landmarks per displayed frame
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y

            x_.append(x)
            y_.append(y)

        x_min, x_max = min(x_), max(x_)
        y_min, y_max = min(y_), max(y_)
        
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            data_aux.append(x - x_min)
            data_aux.append(y - y_min)

        # (x1, y1) is the top left corner of the box
        x1 = int(x_min * W) - 10
        y1 = int(y_min * H) - 10

        # (x2, y2) is the bottom right corner of the box
        x2 = int(x_max * W) - 10
        y2 = int(y_max * H) - 10

        if len(data_aux) == 42:
            predictions = model.predict_proba([np.asarray(data_aux)])
            if max(predictions[0]) >= 0.25:  
                prediction = model.predict([np.asarray(data_aux)])
                predicted_character = chr(int(prediction[0]) + 65) #chr(65) = 'A'
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                cv2.putText(frame, predicted_character, (x2, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
        
    cv2.imshow('frame', frame)

    # Press 'q' to quit program
    if cv2.waitKey(1) == ord('q'): break

cap.release()
cv2.destroyAllWindows()

0.3
0.27
0.51
0.48
0.48
0.44
0.37
0.31
0.31
0.3
0.3
0.43
0.43
0.37
0.36
0.39
0.35
0.35
0.24
0.29
0.29
0.4
0.38
0.38
0.38
0.37
0.35
0.33
0.37
0.37
0.37
0.37
0.37
0.37
0.35
0.37
0.36
0.37
0.37
0.37
0.37
0.37
0.35
0.36
0.34
0.35
0.36
0.34
0.35
0.35
0.37
0.37
0.37
0.36
0.34
0.35
0.37
0.35
0.37
0.37
0.37
0.32
0.35
0.33
0.33
0.33
0.34
0.36
0.32
0.36
0.35
0.34
0.37
0.36
0.38
0.37
0.35
0.37
0.37
0.34
0.34
0.35
0.34
0.33
0.34
0.33
0.37
0.35
0.32
0.39
0.47
0.22
0.21
0.21
0.21
0.21
0.21
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.19
0.19
0.16
0.15
0.15
0.15
0.15
0.15
0.15
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.2
0.2
0.2
0.19
0.19
0.2
0.2
0.19
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.18
0.19
0.19
0.18
0.19
0.19
0.17
0.17
0.19
0.19
0.18
0.2
0.2
0.2
0.2
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.18
0.19
0.2
0.2
0.2
0.2
0.19
0.19